In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Yelp API Signin
with open('/Users/austi/.secret/yelp_api.json') as f:
    login = json.load(f)
# Instantiate YelpAPI
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
LOCATION = "Little Rock, AR" 
TERM = "rock shop"

In [4]:
# Define JSON_FILE
JSON_FILE = "Data/reults_in_progress_LittleRock_AR_rockShop.json"
JSON_FILE

'Data/reults_in_progress_LittleRock_AR_rockShop.json'

In [5]:
# JSON file function
def create_json_file(JSON_FILE,  delete_if_exists=False):
    file_exists = os.path.isfile(JSON_FILE)
    if file_exists == True:
        if delete_if_exists==True:
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            os.remove(JSON_FILE)
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            

    else:
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        folder = os.path.dirname(JSON_FILE)
        if len(folder)>0:
            os.makedirs(folder,exist_ok=True)
        with open(JSON_FILE,'w') as f:
            json.dump([],f)

In [6]:

create_json_file(JSON_FILE, delete_if_exists=True)

with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    

n_results = len(previous_results)
print(f'- {n_results} previous results found.')

results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)

total_results = results['total']

results_per_page = len(results['businesses'])

n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/reults_in_progress_LittleRock_AR_rockShop.json already exists. Deleting previous file...
[i] Data/reults_in_progress_LittleRock_AR_rockShop.json not found. Saving empty list to new file.
- 0 previous results found.


4

In [7]:
for i in tqdm_notebook( range(1,n_pages+1)):
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break

    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)

    previous_results.extend(results['businesses'])

    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,hc8SkNmMXtZdaryo5O0jgw,argenta-bead-company-little-rock-3,Argenta Bead Company,https://s3-media1.fl.yelpcdn.com/bphoto/N5O3DX...,False,https://www.yelp.com/biz/argenta-bead-company-...,11,"[{'alias': 'jewelry', 'title': 'Jewelry'}, {'a...",5.0,"{'latitude': 34.7331779396823, 'longitude': -9...",[],$$,"{'address1': '1608 Main St', 'address2': '', '...",15015370928,(501) 537-0928,7765.738928
1,hFqYM2a1h1zn_VZlwws2Zw,shop-the-rock-little-rock,Shop the Rock,,False,https://www.yelp.com/biz/shop-the-rock-little-...,1,"[{'alias': 'giftshops', 'title': 'Gift Shops'}...",1.0,"{'latitude': 34.74774, 'longitude': -92.26564}",[],NaN,"{'address1': '400 President Clinton Ave', 'add...",15013203515,(501) 320-3515,8260.724074
2,yJDHOwcv9MzeLfyhWmDJZQ,danwerke-jewelers-little-rock,Danwerke Jewelers,https://s3-media3.fl.yelpcdn.com/bphoto/JOmQ_3...,False,https://www.yelp.com/biz/danwerke-jewelers-lit...,9,"[{'alias': 'jewelry', 'title': 'Jewelry'}]",3.5,"{'latitude': 34.7636260986328, 'longitude': -9...",[],$,"{'address1': '9816 N Rodney Parham Rd', 'addre...",15012259593,(501) 225-9593,3028.919763
3,ickdw7j7rSSU9X4XYYGZSQ,moxy-modern-mercantile-little-rock,MOXY Modern Mercantile,https://s3-media2.fl.yelpcdn.com/bphoto/4KvqT4...,False,https://www.yelp.com/biz/moxy-modern-mercantil...,8,"[{'alias': 'vintage', 'title': 'Used, Vintage ...",4.5,"{'latitude': 34.734612, 'longitude': -92.272644}",[],$$,"{'address1': '1419 S Main St', 'address2': '',...",15013742474,(501) 374-2474,7815.919461
4,vqr9xSKAIOEWCk5xjbsTIw,gold-and-diamond-prospectors-little-rock,Gold & Diamond Prospectors,https://s3-media3.fl.yelpcdn.com/bphoto/CLcMs7...,False,https://www.yelp.com/biz/gold-and-diamond-pros...,1,"[{'alias': 'goldbuyers', 'title': 'Gold Buyers...",1.0,"{'latitude': 34.75822, 'longitude': -92.37527}",[],NaN,"{'address1': '9112 N Rodney Parham Rd', 'addre...",15015650065,(501) 565-0065,2063.985235


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
67,zJ85D3Nu9ocVvZcYwkya1g,kroger-food-stores-england,Kroger Food Stores,,False,https://www.yelp.com/biz/kroger-food-stores-en...,1,"[{'alias': 'florists', 'title': 'Florists'}, {...",5.0,"{'latitude': 34.5448719, 'longitude': -91.9569...",[],NaN,"{'address1': '301 Pine Bluff Hwy', 'address2':...",15018422491,(501) 842-2491,43009.589139
68,xyxE4JztaJIh1niRTuPlDQ,fairy-tale-floral-north-little-rock,Fairy Tale Floral,https://s3-media4.fl.yelpcdn.com/bphoto/FhxlzR...,False,https://www.yelp.com/biz/fairy-tale-floral-nor...,17,"[{'alias': 'florists', 'title': 'Florists'}]",2.0,"{'latitude': 34.78577, 'longitude': -92.26084}",[],$$,"{'address1': '3321 John F Kennedy Blvd', 'addr...",15017538393,(501) 753-8393,9677.451563
69,U5pKXaWFYmi5FCLlEJaPMg,american-made-general-store-conway,American Made General Store,https://s3-media4.fl.yelpcdn.com/bphoto/CioFoh...,False,https://www.yelp.com/biz/american-made-general...,1,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",1.0,"{'latitude': 35.086110281811465, 'longitude': ...",[],NaN,"{'address1': '429 US-64 E', 'address2': None, ...",15015857090,(501) 585-7090,37591.120511
70,lAzh7ZO3_VciHd1MTVMbMw,trinkets-and-traditions-flower-shop-little-rock,Trinkets And Traditions Flower Shop,https://s3-media2.fl.yelpcdn.com/bphoto/CJvCL7...,False,https://www.yelp.com/biz/trinkets-and-traditio...,7,"[{'alias': 'florists', 'title': 'Florists'}]",1.0,"{'latitude': 34.6259299, 'longitude': -92.31506}",[],NaN,"{'address1': '13724 Arch St', 'address2': None...",15015039770,(501) 503-9770,14154.709735
71,I9B4lcD8pGO5AatyJwJ3fQ,north-hills-florist-and-gifts-sherwood-2,North Hills Florist & Gifts,https://s3-media4.fl.yelpcdn.com/bphoto/aVZ8Nj...,False,https://www.yelp.com/biz/north-hills-florist-a...,7,"[{'alias': 'florists', 'title': 'Florists'}]",4.0,"{'latitude': 34.81768513802531, 'longitude': -...",[],NaN,"{'address1': '7311 N Hills Blvd', 'address2': ...",15018351367,(501) 835-1367,13562.457256


In [9]:
final_df.duplicated(subset='id').sum()

0

In [10]:
final_df.to_csv('Data/final_results_LittleRock_AR_rockShop.csv.gz', compression='gzip',index=False)

In [13]:
newdf = pd.read_csv('Data/final_results_LittleRock_AR_rockShop.csv.gz')
newdf.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,hc8SkNmMXtZdaryo5O0jgw,argenta-bead-company-little-rock-3,Argenta Bead Company,https://s3-media1.fl.yelpcdn.com/bphoto/N5O3DX...,False,https://www.yelp.com/biz/argenta-bead-company-...,11,"[{'alias': 'jewelry', 'title': 'Jewelry'}, {'a...",5.0,"{'latitude': 34.7331779396823, 'longitude': -9...",[],$$,"{'address1': '1608 Main St', 'address2': '', '...",15015370928,(501) 537-0928,7765.738928
1,hFqYM2a1h1zn_VZlwws2Zw,shop-the-rock-little-rock,Shop the Rock,NaN,False,https://www.yelp.com/biz/shop-the-rock-little-...,1,"[{'alias': 'giftshops', 'title': 'Gift Shops'}...",1.0,"{'latitude': 34.74774, 'longitude': -92.26564}",[],NaN,"{'address1': '400 President Clinton Ave', 'add...",15013203515,(501) 320-3515,8260.724074
2,yJDHOwcv9MzeLfyhWmDJZQ,danwerke-jewelers-little-rock,Danwerke Jewelers,https://s3-media3.fl.yelpcdn.com/bphoto/JOmQ_3...,False,https://www.yelp.com/biz/danwerke-jewelers-lit...,9,"[{'alias': 'jewelry', 'title': 'Jewelry'}]",3.5,"{'latitude': 34.7636260986328, 'longitude': -9...",[],$,"{'address1': '9816 N Rodney Parham Rd', 'addre...",15012259593,(501) 225-9593,3028.919763
3,ickdw7j7rSSU9X4XYYGZSQ,moxy-modern-mercantile-little-rock,MOXY Modern Mercantile,https://s3-media2.fl.yelpcdn.com/bphoto/4KvqT4...,False,https://www.yelp.com/biz/moxy-modern-mercantil...,8,"[{'alias': 'vintage', 'title': 'Used, Vintage ...",4.5,"{'latitude': 34.734612, 'longitude': -92.272644}",[],$$,"{'address1': '1419 S Main St', 'address2': '',...",15013742474,(501) 374-2474,7815.919461
4,vqr9xSKAIOEWCk5xjbsTIw,gold-and-diamond-prospectors-little-rock,Gold & Diamond Prospectors,https://s3-media3.fl.yelpcdn.com/bphoto/CLcMs7...,False,https://www.yelp.com/biz/gold-and-diamond-pros...,1,"[{'alias': 'goldbuyers', 'title': 'Gold Buyers...",1.0,"{'latitude': 34.75822, 'longitude': -92.37527}",[],NaN,"{'address1': '9112 N Rodney Parham Rd', 'addre...",15015650065,(501) 565-0065,2063.985235
